In [1]:
!pip install pulp

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import random
from itertools import permutations
from pulp import LpProblem, LpVariable, LpStatus, LpMinimize, lpSum
from typing import Dict, List, Tuple
from pulp import GLPK_CMD

def generate_dummy_data(n_pois: int, n_days: int) -> tuple:
    """
    Generate dummy data for the itinerary optimization problem.

    Parameters:
    - n_pois: The number of points of interest (POIs).
    - n_days: The number of days.

    Returns:
    - pois: A list of POI names.
    - days: A list of day identifiers.
    - travel_time: A dictionary with travel times between each pair of POIs.
    """
    pois = [f'poi_{i}' for i in range(n_pois)]
    days = [f'D{day}' for day in range(1, n_days + 1)]
    travel_time = {(poi1, poi2): random.randint(10, 60) for poi1, poi2 in permutations(pois, 2)}
    return pois, days, travel_time



In [121]:
# Initialize data
n_pois = 100  # Number of POIs
n_days = 14  # Number of days
n_nonce_nodes = 1 # Number of non-POI nodes

nonce_nodes = [f"nonce_node_{str(i)}" for i in range(n_nonce_nodes)]
pois, days, travel_time = generate_dummy_data(n_pois, n_days)


In [122]:

# Initialize the problem
problem = LpProblem("Itinerary_Optimization", LpMinimize)

# Variables
order_vars = {
    (poi1, poi2, day): LpVariable(f"order_{poi1}_{poi2}_{day}", cat="Binary")
    for poi1 in pois + nonce_nodes for poi2 in pois + nonce_nodes 
    for day in days if poi1 != poi2
}

# Objective function: Minimize the total travel time
problem += lpSum(
    travel_time[(poi_from, poi_to)] * order_vars[(poi_from, poi_to, day)]
    for poi_from, poi_to in permutations(pois, 2) for day in days
)

# Constraints

# Each POI visited exactly once across all days
for poi in pois:
    problem += lpSum(
        order_vars[(poi, other_poi, day)] + order_vars[(other_poi, poi, day)]
        for day in days for other_poi in pois + nonce_nodes if poi != other_poi
    ) == 2, f"enter_and_exit_{poi}"

# Ensure that starting nonce_nodes are visited n_days times
problem += lpSum(order_vars[(nonce_node, poi, day)] 
                 for poi in pois for day in days
                 for nonce_node in nonce_nodes) == n_days, "start_nonce"

# Ensure that ending nonce_nodes are visited n_days times
problem += lpSum(order_vars[(poi, nonce_node, day)] 
                 for poi in pois for day in days
                 for nonce_node in nonce_nodes) == n_days, "end_nonce"

for day in days:
    problem += lpSum(
        order_vars[(nonce_node, poi, day)] 
        for poi in pois for nonce_node in nonce_nodes) == 1, f"start_nonce_{day}"
    
    problem += lpSum(
        order_vars[(poi, nonce_node, day)] 
        for poi in pois for nonce_node in nonce_nodes) == 1, f"end_nonce_{day}"


# Ensure entering and exiting each POI occurs within the same day
for poi in pois:
    for day in days:
        problem += lpSum(
            order_vars[(poi, other_poi, day)]
            for other_poi in pois + nonce_nodes if poi != other_poi
        ) == lpSum(
            order_vars[(other_poi, poi, day)]
            for other_poi in pois + nonce_nodes if poi != other_poi
        ), f"same_day_visit_{poi}_{day}"

# Ensure entering and exiting each nonce node occurs within the same day
for nonce_node in nonce_nodes:
    for day in days:
        problem += lpSum(
            order_vars[(poi, nonce_node, day)]
            for poi in pois
        ) == lpSum(
            order_vars[(nonce_node, poi, day)]
            for poi in pois
        ), f"same_day_nonce_visit_{nonce_node}_{poi}_{day}"


# problem
# Solve the problem
status = problem.solve(GLPK_CMD(msg=0))

# Check the status and print the results
if LpStatus[status] == 'Optimal':
    total_time = 0
    print("Solution Found:\n")
    results = []
    for var in order_vars:
        if order_vars[var].varValue == 1:
            if not var[0].startswith('nonce_node_') and not var[1].startswith('nonce_node_'):
                total_time += travel_time[(var[0], var[1])]
            results.append(var)
            print(f"{var} = {order_vars[var].varValue}")
else:
    print("No optimal solution found.")
print(total_time)

Solution Found:

('poi_0', 'poi_96', 'D1') = 1
('poi_1', 'poi_23', 'D2') = 1
('poi_2', 'poi_72', 'D8') = 1
('poi_3', 'poi_62', 'D8') = 1
('poi_4', 'poi_25', 'D7') = 1
('poi_5', 'poi_4', 'D7') = 1
('poi_6', 'poi_26', 'D8') = 1
('poi_7', 'poi_71', 'D8') = 1
('poi_8', 'poi_17', 'D1') = 1
('poi_9', 'poi_89', 'D5') = 1
('poi_10', 'poi_49', 'D8') = 1
('poi_11', 'poi_88', 'D8') = 1
('poi_12', 'poi_3', 'D8') = 1
('poi_13', 'poi_90', 'D8') = 1
('poi_14', 'poi_12', 'D8') = 1
('poi_15', 'nonce_node_0', 'D12') = 1
('poi_16', 'nonce_node_0', 'D7') = 1
('poi_17', 'poi_34', 'D1') = 1
('poi_18', 'poi_29', 'D12') = 1
('poi_19', 'poi_7', 'D8') = 1
('poi_20', 'poi_92', 'D8') = 1
('poi_21', 'poi_50', 'D13') = 1
('poi_22', 'poi_98', 'D13') = 1
('poi_23', 'poi_53', 'D2') = 1
('poi_24', 'poi_79', 'D9') = 1
('poi_25', 'poi_16', 'D7') = 1
('poi_26', 'poi_33', 'D8') = 1
('poi_27', 'poi_95', 'D13') = 1
('poi_28', 'nonce_node_0', 'D4') = 1
('poi_29', 'poi_18', 'D12') = 1
('poi_30', 'nonce_node_0', 'D9') = 1
('poi

In [53]:
not var[0].startswith('nonce_node_') and not var[1].startswith('nonce_node_') 

False